In [27]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [28]:
if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')

In [29]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [30]:

num_epochs = 100
batch_size = 128
learning_rate = 1e-3

In [31]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

In [32]:
dataset = MNIST('./data', download=True,transform=img_transform)
dataset.data.shape

torch.Size([60000, 28, 28])

In [33]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # batch_size = 128

In [34]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),
            # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),
            # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),
            # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)
            # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),
            # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [35]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [36]:
dataloader.dataset.data.shape

torch.Size([60000, 28, 28])

In [37]:
for epoch in range(num_epochs): #100
    total_loss = 0
    for data in dataloader:
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img) # model(x)
        loss = criterion(output, img) # calculate the loss

        # ===================backward====================
        optimizer.zero_grad() # Adam
        loss.backward() # use loss to do backward
        optimizer.step() # Adam
        total_loss += loss.data
        # calculate the total loss in this epoch

    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, total_loss))

    if epoch % 10 == 0: # do it every 10 iters
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [38]:
torch.cuda.is_available()

True

In [ ]:
torch.save(model.state_dict(), './conv_autoencoder.pth')